In [1]:
import open3d as o3d
import os
import numpy as np
from tqdm import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_matrix_from_kitti_file(line: str) -> np.ndarray:
    matrix = np.eye(4)
    matrix[:3, :4] = np.array(list(map(float, line.rstrip().split(" ")))).reshape(3, 4)

    return matrix


def get_calibration_matrix_from_calib_file(path_to_calibration_file: str) -> np.ndarray:
    with open(path_to_calibration_file) as file:
        return get_matrix_from_kitti_file(file.readlines()[4][4:])


def get_position_matrices_from_poses_file(path_to_poses_file: str) -> list:
    with open(path_to_poses_file) as file:
        lines = file.readlines()
        pose_matrices = []

        for frame_number in range(len(lines)):
            pose_matrices.append(get_matrix_from_kitti_file(lines[frame_number]))

        return pose_matrices
    
def get_point_cloud_from_bin_file(path_to_bin_file: str) -> o3d.geometry.PointCloud:
    """
    Function allows you to get a point cloud from a file with lidar data
    """
    point_cloud_numpy = np.fromfile(path_to_bin_file, dtype=np.float32).reshape(-1, 4)
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(point_cloud_numpy[:, :3])
    return point_cloud

def transform_positions_in_point_cloud(
    calib_matrix: np.ndarray,
    pose_matrix: np.ndarray,
    point_cloud: o3d.geometry.PointCloud
) -> o3d.geometry.PointCloud:
    left_camera_matrix = pose_matrix @ calib_matrix

    return point_cloud.transform(left_camera_matrix)

In [3]:
path_to_bin_dir = "/home/anasyasiia/Downloads/SemanticKITTI_00/"
path_to_poses_file = "/home/anasyasiia/Downloads/rss2018_frame2frame/00.txt"
path_to_calib_file = "/home/anasyasiia/Downloads/data_odometry_calib/dataset/sequences/00/calib.txt"

point_cloud = o3d.geometry.PointCloud()
calib_matrix = get_calibration_matrix_from_calib_file(path_to_calib_file)
poses_matrices = get_position_matrices_from_poses_file(path_to_poses_file)

N = 1000
for index in tqdm(range(750, N + 1)):
    path_to_current_point_cloud = os.path.join(
        path_to_bin_dir, str(index).zfill(6) + ".bin"
    )
    temp_point_cloud = get_point_cloud_from_bin_file(path_to_current_point_cloud)
    temp_point_cloud = transform_positions_in_point_cloud(
        calib_matrix, poses_matrices[index], temp_point_cloud
    )

    if not index % 5:
        point_cloud = point_cloud + temp_point_cloud
#     if index % 50:
#         point_cloud = point_cloud.voxel_down_sample(0.05)

100%|██████████| 251/251 [00:20<00:00, 11.96it/s]


In [4]:
point_cloud_ds = point_cloud.voxel_down_sample(0.2)

In [5]:
o3d.visualization.draw_geometries([point_cloud_ds])

In [6]:
o3d.io.write_point_cloud('test_750_1000_20cm.pcd', point_cloud_ds)

True

In [39]:
import matplotlib.pyplot as plt

def visualize_labels(pcd, labels):
    geometries = []
    points = np.asarray(pcd.points)
    N_planes = np.max(labels) + 1
    
    cmap = plt.cm.get_cmap('Spectral')
    
    for i in range(N_planes):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points[np.where(labels == i)])
        print(cmap(i / N_planes))
        pcd.paint_uniform_color(cmap(i / N_planes)[:3])
        geometries.append(pcd)

    o3d.visualization.draw_geometries(geometries)

In [71]:
visualize_labels(point_cloud_ds, labels)

(0.6196078431372549, 0.00392156862745098, 0.25882352941176473, 1.0)
(0.998077662437524, 0.9992310649750096, 0.7460207612456747, 1.0)


In [26]:
point_cloud_ds

array([[-197.03268227,   15.03829018,  310.90641669],
       [-189.22224677,   14.43441436,  309.35343541],
       [-189.26649891,   14.43330104,  309.12879376],
       ...,
       [-188.94523831,   13.14214186,  313.07322507],
       [-102.49077192,   13.38577921,  385.03804255],
       [ -82.98365807,    9.69436086,  339.98985685]])

In [67]:
index = 750
path_to_current_point_cloud = os.path.join(
        path_to_bin_dir, str(index).zfill(6) + ".bin"
    )
temp_point_cloud = get_point_cloud_from_bin_file(path_to_current_point_cloud)
temp_point_cloud = transform_positions_in_point_cloud(
    calib_matrix, poses_matrices[index], temp_point_cloud
    )

obs_points = np.asarray(temp_point_cloud.points)
temp_point_cloud.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([point_cloud_ds, temp_point_cloud])

In [45]:
map_tree = o3d.geometry.KDTreeFlann(point_cloud_ds)

In [68]:
cmap = plt.cm.get_cmap('Spectral')
N_planes = np.max(labels) + 1
for point_idx in range(obs_points.shape[0]):
    [k, idx, _] = map_tree.search_knn_vector_3d(obs_points[point_idx], 1)
    np.asarray(temp_point_cloud.colors)[point_idx] = cmap(labels[idx[0]] / N_planes)[:3]

In [73]:
o3d.visualization.draw_geometries([temp_point_cloud])

In [72]:
o3d.visualization.draw_geometries([point_cloud_ds])

In [86]:
def remap_labels(pc_obs, pc_map, labels_map):
    cmap = plt.cm.get_cmap('Spectral')
    N_planes = np.max(labels) + 1
    map_tree = o3d.geometry.KDTreeFlann(pc_map)
    obs_points = np.asarray(pc_obs.points)
    pc_obs.paint_uniform_color([0, 0, 0])

    labels_obs = np.zeros(obs_points.shape[0])
    for point_idx in range(obs_points.shape[0]):
        [k, idx, _] = map_tree.search_knn_vector_3d(obs_points[point_idx], 1)
        np.asarray(pc_obs.colors)[point_idx] = cmap(labels_map[idx[0]] / N_planes)[:3]
        labels_obs[point_idx] = labels_map[idx[0]]
        
    o3d.visualization.draw_geometries([pc_obs])
    return labels_obs

In [87]:
index = 900
path_to_current_point_cloud = os.path.join(
        path_to_bin_dir, str(index).zfill(6) + ".bin"
    )
temp_point_cloud = get_point_cloud_from_bin_file(path_to_current_point_cloud)
temp_point_cloud = transform_positions_in_point_cloud(
    calib_matrix, poses_matrices[index], temp_point_cloud
    )

# o3d.visualization.draw_geometries([temp_point_cloud])

obs_labels = remap_labels(temp_point_cloud, point_cloud_ds, labels)

array([0., 0., 0., ..., 1., 1., 1.])

In [1]:
pc5 = o3d.io.read_point_cloud('pcds/test_750_1000_20cm.pcd')

NameError: name 'o3d' is not defined